In [ ]:
####################################
####################################
####################################
###############
###############
###############
# PART II: Autoencoders
###############
###############
###############
####################################
####################################
####################################

####################################
### let's build an autoencoder
tf.reset_default_graph()
batch_size = 100
data_tf = tf.placeholder(shape=[None, data.shape[1]], dtype=tf.float32, name='data_tf')


# layers will be input -> 100 -> 2 --> 100 -> output
hidden_layer1_tf = layer(data_tf, 100, 'hidden_layer1', activation=tf.nn.relu)
hidden_layer2_tf = layer(hidden_layer1_tf, 2, 'hidden_layer2', activation=None)
hidden_layer3_tf = layer(hidden_layer2_tf, 100, 'hidden_layer3', activation=tf.nn.relu)
output_tf = layer(hidden_layer3_tf, data.shape[1], 'output_tf', activation=None)


# use mean-squared-error reconstruction loss
loss_tf = tf.reduce_mean((data_tf - output_tf)**2)

# this part is all the same as before
learning_rate = .001
opt = tf.train.AdamOptimizer(learning_rate)

train_op = opt.minimize(loss_tf)

sess = tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=.1)))
sess.run(tf.global_variables_initializer())

for epoch in range(10):
    random_order = np.random.choice(data.shape[0], data.shape[0], replace=False)
    data_randomized = data[random_order]

    for data_batch in np.array_split(data_randomized, data_randomized.shape[0] // batch_size):
        if step % 100 == 0:
            loss_np = sess.run(loss_tf, {data_tf: data_batch})
            print("Step: {} Loss: {:.3f}".format(step, loss_np))
        step += 1

        sess.run(train_op, {data_tf: data_batch})
###
####################################



####################################
### rather than evaluating our model with our data like
### we did with the classifier, we can now use our model
### to evaluate our data (aka exploratory data analysis)!


# let's get the 2D internal hidden layer and visualize it
# with a scatter plot
viz_coordinates = []
for data_batch in np.array_split(data, data.shape[0] // batch_size):
    out = sess.run(hidden_layer2_tf, {data_tf: data_batch})
    viz_coordinates.append(out)
viz_coordinates = np.concatenate(viz_coordinates, axis=0)


fig = plt.figure()
ax = fig.subplots(1, 1)
ax.scatter(viz_coordinates[:, 0], viz_coordinates[:, 1], c=labels, s=5)

ax.set_xlabel('AE Coordinate 1')
ax.set_ylabel('AE Coordinate 2')

###
####################################





####################################
### Exercise 4

# notice we used activation=None for the hidden layer we were going to visualize
# repeat the process with other activation functions like tf.nn.relu, tf.nn.sigmoid, tf.nn.tanh, etc...
# and note how the visualization changes. has the data changed at all?

###
####################################



####################################
### Exercise 5

# now turn the activation for the visualization layer back to None, but experiment with
# the activation function for the 100-dimensional layers. is there a change? why?

###
####################################